In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk.util import ngrams
import plotly.express as px
import scipy.stats
import math
import statistics
import string 
import re
from collections import defaultdict
from gensim import corpora, models
from nltk.text import Text
# ----------------------------------------
import random
from nltk.tokenize import word_tokenize, sent_tokenize
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

# ----------------------------------------
print('Success!')

Success!


### Import all the data 

In [2]:
# data importing 
lowes_6026 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_3286026.csv')
lowes_0314 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_3380314.csv')
lowes_0511 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_50280511.csv') 
lowes_6162 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_1003066162.csv')
lowes_6209 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_5000026209.csv')
lowes_9855 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_5001899855.csv')
# data importing 
homedepot_4796 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\homedepot.com_204074796.csv')
homedepot_7071 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\homedepot.com_319247071.csv')

In [3]:
# set display option to max to see all information in review column
pd.set_option('display.max_colwidth', -1)

C:\Users\lxf12\AppData\Local\Temp\ipykernel_15484\779606820.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [4]:
# drop null value
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]
namelist = ['lowes_6026', 'lowes_0314', 'lowes_0511', 'lowes_6162',
                    'lowes_6209', 'lowes_9855', 
                     'homedepot_4796', 'homedepot_7071']
n = 0 

for i in datalist: 
    # drop rows with Not a Number (NaN) and None values
    name = namelist[n]
    n += 1
    print(name + ' has null rows amount: '+ str(i['Review'].isna().sum()))
    i.dropna(subset=['Review'], inplace=True)
    print('after cleaning, ' + name + ' has null rows amount: ' 
          + str(i['Review'].isna().sum()))

lowes_6026 has null rows amount: 30
after cleaning, lowes_6026 has null rows amount: 0
lowes_0314 has null rows amount: 10
after cleaning, lowes_0314 has null rows amount: 0
lowes_0511 has null rows amount: 15
after cleaning, lowes_0511 has null rows amount: 0
lowes_6162 has null rows amount: 297
after cleaning, lowes_6162 has null rows amount: 0
lowes_6209 has null rows amount: 76
after cleaning, lowes_6209 has null rows amount: 0
lowes_9855 has null rows amount: 170
after cleaning, lowes_9855 has null rows amount: 0
homedepot_4796 has null rows amount: 186
after cleaning, homedepot_4796 has null rows amount: 0
homedepot_7071 has null rows amount: 9
after cleaning, homedepot_7071 has null rows amount: 0


In [5]:
# lowes_0511.head(2)

In [6]:
# homedepot_7071.head(2)

In [7]:
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]

# Loop through the list and print the name and shape of each dataframe
for i in datalist:
    print("DataFrame:", type(i).__name__)
    print("Number of rows:", i.shape[0])

DataFrame: DataFrame
Number of rows: 780
DataFrame: DataFrame
Number of rows: 390
DataFrame: DataFrame
Number of rows: 25
DataFrame: DataFrame
Number of rows: 203
DataFrame: DataFrame
Number of rows: 24
DataFrame: DataFrame
Number of rows: 100
DataFrame: DataFrame
Number of rows: 324
DataFrame: DataFrame
Number of rows: 51


### Combine the 8 data at once 

In [8]:
# Combine the dataframes by rows
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]
combined_df = pd.concat(datalist, axis=0)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1897 entries, 0 to 59
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Market_place  1897 non-null   object 
 1   URL           1897 non-null   object 
 2   Product_name  1897 non-null   object 
 3   SKU           1897 non-null   object 
 4   Price         1897 non-null   float64
 5   Currency      1897 non-null   object 
 6   Subject       1834 non-null   object 
 7   Author        1855 non-null   object 
 8   Date          1897 non-null   object 
 9   Review        1897 non-null   object 
 10  Stars         1897 non-null   int64  
 11  like          1522 non-null   float64
 12  dislike       1522 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 207.5+ KB


In [9]:
# combined_df.head(2)

In [10]:
# combined_df.tail(2)

In [11]:
# combined_df

### Separate Negative and Positive Ranking

In [12]:
# 8 data x 2 categories 
try:
    user_select = int(input("Please select negative(0) or positive ranking(1): "))
except: 
    print("Your input is not correct. Please input again according the tips.")

if user_select == 0:
    # get reviews that have given 1, 2, and 3 stars (negative)
    combined_df = combined_df.loc[combined_df['Stars'] <= 3]
elif user_select == 1:
    # get reviews that have given 4 and 5 stars (positive)
    combined_df = combined_df.loc[combined_df['Stars'] > 3]
else: 
    print("Your input is not correct. Please input again according the tips.")

Please select negative(0) or positive ranking(1): 1


In [13]:
# combined_df

### Extract the review text

In [14]:
# make it all string
text_01 = combined_df['Review'].to_string(index=False)
# text_01

In [15]:
type(text_01)

str

In [16]:
text_02 = text_01

In [17]:
# convert all characters to lowercase
text_02 = text_02.lower()
# Create a translation table with all punctuation characters mapped to None
translator = str.maketrans('', '', string.punctuation)
# remove punctuations using RegexpTokenizer
text_02 = text_02.translate(translator)
# Use regex to remove all numbers
text_02 = re.sub(r'\d+', '', text_02)

In [18]:
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    # customize stop_words by including a custom list of words
    custom_stop_words = ['toilet', 'rating', 'provided', 'verified', 'purchaser']
#     Rating provided by a verified purchaser  
# #     custom_stop_words = ['new','toilet', 'factory', 'flush', 'sku', 'photos', 'stay', 
# #                                      'lid', 'good', 'great', 'handle', 'top', 'high', 'urine']
    stop_words.update(custom_stop_words)
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lxf12\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
text_02 = remove_stopwords(text_02)

In [20]:
# remove punctuations using RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
token = tokenizer.tokenize(text_02)

### N-gram

In [21]:
# N-gram generation
N = int(input('How many words do you want to aggregate? (1-8)'))
N_grams = list(ngrams(token, N))
words = [' '.join(tr) for tr in N_grams]

How many words do you want to aggregate? (1-8)4


### Frequency Analysis

In [22]:
# select the data 
# n = int(input('Which one do you want to look at? (1-8)'))

word_frequency = FreqDist(N_grams)

# # Identify co-occurring trigrams
# cooccurring_trigrams = []
# for N_grams in word_frequency:
#     if word_frequency[N_grams] > 1:
#         cooccurring_trigrams.append(N_grams)

# word_frequency = FreqDist(cooccurring_trigrams)        
word_frequency.most_common(10)
# word_frequency.plot(30,cumulative=False)
# plt.show()

[(('review', 'collected', 'part', 'promotion'), 16),
 (('uses', 'much', 'less', 'water'), 6),
 (('works', 'well', 'easy', 'install'), 6),
 (('easy', 'install', 'works', 'great'), 6),
 (('easy', 'install', 'looks', 'great'), 5),
 (('except', 'water', 'supply', 'line'), 5),
 (('new', 'water', 'supply', 'line'), 5),
 (('less', 'water', 'old', 'one'), 4),
 (('adjust', 'water', 'level', 'tank'), 4),
 (('well', 'uses', 'little', 'water'), 4)]

### Topic Modeling
Latent Dirichlet Allocation (LDA), which is a popular unsupervised learning algorithm used for topic modeling.

In [27]:
words = [' '.join(tr) for tr in N_grams]
# Step 2: Create a dictionary of words and convert the text into a bag-of-words format
dictionary = corpora.Dictionary([words])
corpus = [dictionary.doc2bow([word]) for word in words]

# Step 3: Apply LDA to the corpus to identify topics in the text
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

# Step 4: Print the topics and the top words associated with each topic
for i, topic in lda_model.show_topics(formatted=True, num_topics=5, num_words=10):
    print("Topic {}: {}".format(i, topic))

Topic 0: 0.005*"review collected part promotion" + 0.001*"works well easy install" + 0.001*"standard reliant piece gpf" + 0.001*"highly recommend review collected" + 0.001*"piece gpf single flush" + 0.001*"recommend review collected part" + 0.001*"american standard reliant piece" + 0.001*"collected part promotion really" + 0.001*"collected part promotion reliant" + 0.001*"collected part promotion replaced"
Topic 1: 0.001*"bathroom replace old one" + 0.001*"well easy install perfect" + 0.000*"installed easy looks nice" + 0.000*"need except supply line" + 0.000*"easy install took hour" + 0.000*"good product better standard" + 0.000*"good easy install issues" + 0.000*"bolts recommend small medium" + 0.000*"wont lower might cool" + 0.000*"person replace old make"
Topic 2: 0.001*"easy install works great" + 0.001*"reliant piece gpf single" + 0.001*"part promotion installed guest" + 0.001*"gpf single flush elongated" + 0.001*"single flush elongated white" + 0.001*"promotion installed guest b

### Text Summarization
#### In summary, while topic modeling aims to uncover the underlying themes and topics in a large corpus of text, text summarization aims to provide a condensed version of a longer text, capturing the essential information and key points.
#### Two different approaches are used for Text Summarization
1. Extractive Summarization: In Extractive Summarization, we identify essential phrases or sentences from the original text and extract only these phrases from the text. These extracted sentences would be the summary.
2. Abstractive Summarization: We work on generating new sentences from the original text in the Abstractive Summarization approach. The abstractive method contrasts the approach described above, and the sentences generated through this approach might not even be present in the original text.

In [51]:
stopWords = set(stopwords.words("english"))
# words = word_tokenize(result_text_1)
freqTable = dict()
for word in token:
    word = word.lower()
    if word in stopWords:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1

text_03 = remove_stopwords(text_01)
sentences = sent_tokenize(text_03)
# sentences = token
sentenceValue = dict()
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq
                
sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]
    
average = int(sumValues / len(sentenceValue))

In [55]:
# Storing sentences into our summary.
summary = ''
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (6.0 * average)):
        summary += " " + sentence

print(summary)

 Easy install. easy install. easy install. Easy install. easy install. Easy installation. little taller normal toilets issue, would highly recommend price easy install,instruction right on,quality product,after install looked amount water used looked like little,but worked great.They pretested.Will buy another purchased today - installation breeze, however could adjust fill valve leaking, luckily spare fluidmaster fill valve... far flushing good .... needed new somehow tank old one got crack water spilled everywhere work. Easy install. Easy install. Easy installation. Easy install. easy install. easy install. easy install. Easy install. Easy install. Easy installation. Easy install. easy install. easy install. Easy install. Easy install. Easy install. easy install. Easy install. Easy install. Easy install. easy install. Easy install. Easy install. easy install. Easy install. Easy install. Easy install. Easy installation. Easy install. Easy install. easy install. Easy installation. Easy

#### New Methond on Text Summarization
Compute a score for each sentence based on its importance to the overall meaning of the text. The score can be based on various criteria, such as word frequency, sentence length, or semantic similarity to other sentences in the text.

In [56]:
text_03 = remove_stopwords(text_01)

In [58]:
parser = PlaintextParser.from_string(text_01, Tokenizer("english"))
summarizer = LexRankSummarizer()
summary = summarizer(parser.document, sentences_count=8)

for sentence in summary:
    print(sentence)

This is a good toilet.
It's a little taller than normal toilets but not an issue, I would highly recommend this toilet for the price Very easy to install,instruction where right on,quality product,after install looked at the amount of water used looked like it was to little,but it worked great.They are all pretested.Will buy another purchased today - installation was a breeze, however could not adjust the fill valve because it was leaking, luckily i had a spare fluidmaster toilet fill valve... so far it is flushing good .... i needed a new toilet because somehow the tank in my old one got a crack in it and water spilled everywhere while i was at work.
The toilet was fine but the toilet seat was cheap and I had to buy a new one Not hard at all to install.
The only issue I have with this toilet is the bowl does not have a lot of water in it I’ve tried everything but it still has very little water after you flush Well, the height is very nice, and the look and flush - even the lid (plasti

### Context Analysis
#### Check out the context of words having highest frequency

In [59]:
# pos and lemmatizer

lemmatizer = WordNetLemmatizer()
lemm = lemmatizer.lemmatize(text_03)
pos_tag_1 = pos_tag(word_tokenize(lemm))
# pos_tag_1

### select Adjective or Noun 
result_adj_1 = []
for token, tag in pos_tag_1:
    if tag[0:1].lower() == "j":   ### j represents Adj. n represents noun
        result_adj_1.append(token)
print(f"Total number of ADJ: {len(result_adj_1)}\n")
# print(result_adj_1)

Total number of ADJ: 7142



In [60]:
# see each Adjective frequence
ADJ_f_1 = FreqDist(result_adj_1)
# ADJ_f_1
print(f"The top 10 most common Adjective. word appear in review is: {ADJ_f_1.most_common(10)}")

The top 10 most common Adjective. word appear in review is: [('great', 342), ('easy', 341), ('good', 274), ('old', 241), ('flush', 153), ('new', 149), ('little', 127), ('American', 110), ('install', 103), ('small', 98)]


In [61]:
# pos and lemmatizer

lemmatizer = WordNetLemmatizer()
lemm = lemmatizer.lemmatize(text_03)
pos_tag_1 = pos_tag(word_tokenize(lemm))
# pos_tag_1

### select Adjective or Noun 
result_adj_1 = []
for token, tag in pos_tag_1:
    if tag[0:1].lower() == "n":   ### j represents Adj. n represents noun
        result_adj_1.append(token)
print(f"Total number of noun: {len(result_adj_1)}\n")
# print(result_adj_1)

Total number of noun: 13977



In [62]:
# see each noun frequence
noun_f_1 = FreqDist(result_adj_1)
print(f"The top 10 most common Noun. word appear in review is: {noun_f_1.most_common(10)}")

The top 10 most common Noun. word appear in review is: [('water', 494), ('seat', 382), ('toilet', 357), ('install', 253), ('toilets', 232), ('flush', 214), ('tank', 197), ('bathroom', 188), ('price', 187), ('product', 162)]


In [63]:
# pos and lemmatizer

lemmatizer = WordNetLemmatizer()
lemm = lemmatizer.lemmatize(text_03)
pos_tag_1 = pos_tag(word_tokenize(lemm))
# pos_tag_1

### select Adjective or Noun 
result_adj_1 = []
for token, tag in pos_tag_1:
    if tag[0:1].lower() == "v":   ### j represents Adj. n represents noun
        result_adj_1.append(token)
print(f"Total number of verb: {len(result_adj_1)}\n")
# print(result_adj_1)

Total number of verb: 6621



In [64]:
# see each noun frequence
noun_f_1 = FreqDist(result_adj_1)
print(f"The top 10 most common Verb. word appear in review is: {noun_f_1.most_common(10)}")

The top 10 most common Verb. word appear in review is: [('installed', 164), ('bought', 127), ('works', 126), ('replace', 122), ('get', 116), ('needed', 115), ('purchased', 99), ('replaced', 93), ('included', 87), ('flushing', 81)]


#### New Method to Get Context of Key Words
using original text of reviews and locate the full sentence, then sample them to read

In [65]:
nltk.download('punkt')  # Download the Punkt tokenizer

# result_text_1 = datalist[n-1]['Review'].to_string(index=False)
# result_text_1 = text[n-1]
keyword = "seat"
max_sentence_length = 150
num_samples = 10

sentences = nltk.sent_tokenize(text_01)
matching_sentences = []

for sentence in sentences:
    if keyword in sentence:
        if len(sentence) <= max_sentence_length:
            matching_sentences.append(sentence)

if len(matching_sentences) >= num_samples:
    samples = random.sample(matching_sentences, num_samples)
    for sample in samples:
        print(sample)
else:
    print("Not enough matching sentences.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lxf12\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The seat is not the best so that kept us from giving it a 5 star rating.
But I had a new toilet seat to install.
Love the 17.5 height from floor to seat and the water efficient flushing action.
The seat is flimsy but the rest of it is constructed well.
The toilet seat is a little flimsy though.
I went back to Home Depot and the manager let me get a different seat that was made better.
But the seat is very flimsy.
My only complaint has to do with a toilet seat.
I love the slow close seat.
Kind of a strange feel, but easily remedied with a new seat, if it bothers you enough.


locate the fixed length context, then sample them to read

In [66]:
# result_text_1 = datalist[n-1]['Review'].to_string(index=False)
# result_text_1 = text[n-1]
keyword = "seat"
context_length = 100
num_samples = 10

pattern = re.compile(re.escape(keyword))

matches = [(match.start(), match.end()) for match in pattern.finditer(text_01)]

matching_contexts = []
for start, end in matches:
    context_start = max(0, start - context_length)
    context_end = min(len(text_01), end + context_length)
    context = text_01[context_start:end] + text_01[end:context_end]
    matching_contexts.append(context)

if len(matching_contexts) >= num_samples:
    samples = random.sample(matching_contexts, num_samples)
    for sample in samples:
        print(sample)
else:
    print("Not enough matching contexts.")

                                                                                   
Says slow close seat and it definitely isn’t, should b corrected                                                        
   
American Standard has a winner in this little beauty. Easy to install (did it myself except for seating the toilet on the wax ring, then had my husband do that). Perfect height, all the accessories to
e side. I like that better. Easier to spray with disinfectant when cleaning. It comes with a toilet seat that goes slow as it auto closes. But you do have to sort of start the motion or it won't lower by 
ilets. Sits taller than standard ones and uses less water. Never an issue flushing. I would say the seat isnt as comfortable as my last one                                                                 
t and the elongated toilet. The only thing I do not like about the toilet is the cheap hard plastic seat that is on it. I feel that American Standard should replace the seat with a

#### Text Summarization on the Matching sentences 

In [67]:
matching_sentences = ' '.join(matching_sentences)

In [69]:
stopWords = set(stopwords.words("english"))
words = word_tokenize(matching_sentences)
freqTable = dict()
for word in words:
    word = word.lower()
    if word in stopWords:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1

sentences = sent_tokenize(matching_sentences)
sentenceValue = dict()
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq
                
sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]
    
average = int(sumValues / len(sentenceValue))

In [82]:
# Storing sentences into our summary.
summary = ''
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.28 * average)):
        summary += " " + sentence
print(summary)

 The toilet seat is cheap but fortunately the old one, purchased new last year, fits well. Also, the toilet seat is rather cheaply made, but I bought a nickel brushed hinge toilet seat that is slow close and that fixed that problem. We too replaced the plastic seat with a $10 wooden seat, but we really like the toilet. The toilet seat is a little flimsy but, I do like the way it closes slowly and quietly. The only (slight) downside is the plastic toilet seat that came with it - it is not real sturdy, but is adequate. As many of the other reviewers noted, the seat and lid of this toilet are flimsy and paper-thin plastic. The cheap, white seat that's included enables A.S. to keep the price of the toilet down -- and surely, this toilet is a bargain. From my perspective, at least, I'd rather they kept the price down than provide me with a better toilet seat that I wouldn't use. I didn't use the plastic seat that comes with the toilet, like the Church seats Lowes sells much better.


In [83]:
parser = PlaintextParser.from_string(matching_sentences, Tokenizer("english"))
summarizer = LexRankSummarizer()
summary = summarizer(parser.document, sentences_count=8)

for sentence in summary:
    print(sentence)

The seat is very uncomfortable however so I will be buying a new soft close seat that is hopefully more comfortable to sit on..
I used a seat I had just bought for our old toilet as the one that comes in the box is a standard low quality type.
The only thing I did not like was the cheap seat and lid that came with it.
The only thing I do not like about the toilet is the cheap hard plastic seat that is on it.
I really like this toilet, except that the seat is a little flimsy.
I replaced the seat that came with my toilet.
And I like the seat that comes with it.
But the seat is very flimsy.
